In [1]:
from dabry.problem import NavigationProblem
from dabry.wind import DiscreteWind
from dabry.model import ZermeloGeneralModel
from dabry.misc import Utils
from dabry.ddf_manager import DDFmanager
from dabry.solver_ef import SolverEF
from dabry.solver_es import SolverES
import numpy as np
from tqdm.notebook import tqdm

In [2]:
i_member = 0
nm = 51

x_init = Utils.DEG_TO_RAD * np.array((50, -40))
x_target = Utils.DEG_TO_RAD * np.array((0, -48))

bl = Utils.DEG_TO_RAD * np.array((-10, -60))
tr = Utils.DEG_TO_RAD * np.array((60, -25))

with tqdm(total=nm) as pbar:
    for i_member in range(nm):
        wind = DiscreteWind()
        wind.load(f'../output/wind_scenarii3_{i_member}/_wind.h5')
        model = ZermeloGeneralModel(23, coords='gcs')
        model.update_wind(wind)

        pb = NavigationProblem(model, x_init, x_target, 'gcs', bl=bl, tr=tr)

        ddf = DDFmanager(cache_wind=False)
        ddf.setup()
        ddf.set_case(f'wind_scenarii3_{i_member}')
        ddf.clean_output_dir()
        ddf.dump_wind(pb.model.wind)
        
        

        #solver = SolverEF(pb, max_time=20*3600, max_steps=500)
        #res = solver.solve()

        #ddf.dump_trajs([res.traj])
        #ddf.dump_trajs(solver.get_trajs())
        solver = SolverES(pb, rel_tgt_thr=0.02)
        traj = solver.solve(110*3600, quiet=True, dtheta=(50, 'deg'), N_disc=400)
        ddf.dump_trajs([traj])
        ddf.log(pb)
        pbar.update(1)

  0%|          | 0/51 [00:00<?, ?it/s]

No solution found
No solution found
No solution found


In [3]:
import shutil
import os
dir_all = f'../output/wind_scenarii3_all'
if os.path.exists(dir_all):
    shutil.rmtree(dir_all)
os.mkdir(dir_all)
for i_member in range(51):
    try:
        shutil.copy(f'../output/wind_scenarii3_{i_member}/trajectories.h5',
                   f'../output/wind_scenarii3_all/trajectories_{i_member}.h5')
    except FileNotFoundError:
        pass
shutil.copy('../output/wind_scenarii3_0/params.json', '../output/wind_scenarii3_all/params.json')
shutil.copy('../output/wind_scenarii3_0/wind.h5', '../output/wind_scenarii3_all/wind.h5')

'../output/wind_scenarii3_all/wind.h5'

In [15]:
wind = DiscreteWind()
wind.load(f'../output/wind_scenarii_all/wind.h5')
model = ZermeloGeneralModel(23, coords='gcs')
model.update_wind(wind)

x_init = Utils.DEG_TO_RAD * np.array((-55, 40))
x_target = Utils.DEG_TO_RAD * np.array((-10, 40))

bl = Utils.DEG_TO_RAD * np.array((-60, 33))
tr = Utils.DEG_TO_RAD * np.array((-5, 55))

pb = NavigationProblem(model, x_init, x_target, 'gcs', bl=bl, tr=tr)

ddf = DDFmanager(cache_wind=True)
ddf.setup()
ddf.set_case(f'wind_scenarii_all')

from dabry.feedback import HTargetFB
from dabry.stoppingcond import DistanceSC
pb.load_feedback(HTargetFB(x_target, 'gcs'))
sc = DistanceSC(lambda x: (x - x_target) @ (x - x_target), (x_init - x_target) @ (x_init - x_target) / 500)
pb.integrate_trajectory(x_init, sc, t_init=pb.model.wind.t_start, int_step=3600/100)
ddf.dump_trajs(pb.trajs, filename='trajectories_heuristic.h5')
ddf.log(pb)

## Deviation wind

In [13]:
winds = [DiscreteWind() for i_member in range(51)]
for i in range(51):
    winds[i].load(f'../output/wind_scenarii3_{i}/wind.h5')
dev_wind = DiscreteWind()
dev_wind.load(f'../output/wind_scenarii3_0/wind.h5')
with tqdm(total=dev_wind.nx*dev_wind.ny*dev_wind.nt) as pbar:
    for i in range(dev_wind.nx):
        for j in range(dev_wind.ny):
            for k in range(dev_wind.nt):
                mean_u = sum([wind.uv[k, i, j, 0] for wind in winds])/51
                mean_v = sum([wind.uv[k, i, j, 1] for wind in winds])/51
                dev_wind.uv[k, i, j, 0] = np.sqrt(sum([(wind.uv[k, i, j, 0] - mean_u)**2 for wind in winds])/51)
                dev_wind.uv[k, i, j, 1] = np.sqrt(sum([(wind.uv[k, i, j, 1] - mean_v)**2 for wind in winds])/51)
                pbar.update(1)
ddf = DDFmanager()
ddf.setup()
ddf.set_case(f'wind_scenarii3_dev')
ddf.clean_output_dir()
ddf.dump_wind(dev_wind)

  0%|          | 0/456750 [00:00<?, ?it/s]